In [ ]:
# | default_exp _server

In [ ]:
# | export

import asyncio
import multiprocessing
import platform
import signal
import threading
from contextlib import contextmanager
from typing import *
from types import FrameType

import asyncer
import typer

from fastkafka._components.helpers import _import_from_string
from fastkafka._components.logger import get_logger
from fastkafka._components._subprocess import terminate_asyncio_process

In [ ]:
import os
from time import sleep

from pydantic import BaseModel

from fastkafka._components.logger import suppress_timestamps
from fastkafka._components.test_dependencies import generate_app_in_tmp
from fastkafka.testing import ApacheKafkaBroker

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__, level=20)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


class ServerProcess:
    def __init__(self, app: str, kafka_broker_name: str):
        """
        Represents a server process for running the FastKafka application.

        Args:
            app (str): Input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.
            kafka_broker_name (str): The name of the Kafka broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastKafka class.
        """
        self.app = app
        self.should_exit = False
        self.kafka_broker_name = kafka_broker_name

    def run(self) -> None:
        """
        Runs the FastKafka application server process.
        """
        return asyncio.run(self._serve())

    async def _serve(self) -> None:
        """
        Internal method that runs the FastKafka application server.
        """
        self._install_signal_handlers()

        self.application = _import_from_string(self.app)
        self.application.set_kafka_broker(self.kafka_broker_name)

        async with self.application:
            await self._main_loop()

    def _install_signal_handlers(self) -> None:
        """
        Installs signal handlers for handling termination signals.
        """
        if threading.current_thread() is not threading.main_thread():
            raise RuntimeError()

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )
        if platform.system() == "Windows":
            HANDLED_SIGNALS = (*HANDLED_SIGNALS, signal.SIGBREAK) # type: ignore

        def handle_windows_exit(signum: int, frame: Optional[FrameType]) -> None:
            self.should_exit = True

        def handle_exit(sig: int) -> None:
            self.should_exit = True

        for sig in HANDLED_SIGNALS:
            if platform.system() == "Windows":
                signal.signal(sig, handle_windows_exit)
            else:
                loop.add_signal_handler(sig, handle_exit, sig)

    async def _main_loop(self) -> None:
        """
        Main loop for the FastKafka application server process.
        """
        while not self.should_exit:
            await asyncio.sleep(0.1)

In [ ]:
# | export

_app = typer.Typer()


@_app.command()
def run_fastkafka_server_process(
    app: str = typer.Argument(
        ...,
        help="Input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
    kafka_broker: str = typer.Option(
        ...,
        help="Kafka broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastKafka class.",
    ),
) -> None:
    ServerProcess(app, kafka_broker).run()

In [ ]:
# | notest

print("WARNING: make sure you save the notebook before running this cell\n")

print("Exporting and installing the new version of the CLI command...")
await asyncio.create_subprocess_exec("nbdev_export")
export_process = await asyncio.create_subprocess_exec("nbdev_export")
await export_process.wait()
assert export_process.returncode == 0

install_process = await asyncio.create_subprocess_exec(
    "pip", "install", "-e", "..[all]"
)
await install_process.wait()
assert install_process.returncode == 0

print("ok")


Exporting and installing the new version of the CLI command...
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///work/fastkafka
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Attempting uninstall: fastkafka
    Found existing installation: fastkafka 0.7.0.dev0
    Uninstalling fastkafka-0.7.0.dev0:
      Successfully uninstalled fastkafka-0.7.0.dev0
  Running setup.py develop for fastkafka
ok



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
listener_port = 9099
with generate_app_in_tmp() as app:
    async with ApacheKafkaBroker(listener_port=listener_port) as bootstrap_server:
        os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(
            ":"
        )
        proc = await asyncio.create_subprocess_exec(
            "run_fastkafka_server_process",
            "--kafka-broker",
            "localhost",
            app,
            stdout=asyncio.subprocess.PIPE,
        )
        sleep(5)
        await terminate_asyncio_process(proc)
        outputs, _ = await proc.communicate()

        print(outputs.decode("utf-8"))
        expected_returncode = 15 if platform.system() == "Windows" else 0
        assert proc.returncode == expected_returncode, f"{proc.returncode=}"

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
stdout=, stderr=, returncode=1
[INFO] fastkafka._testing.apache_kafka_broker: zookeeper startup falied, generating a new port and retrying...
[INFO] fastkafka._testing.apache_kafka_broker: port=40373
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9099
[INFO] __main__: terminate_asyncio_process(): Terminating the process 5190...
[INFO] __main__: terminate_asyncio_process(): Process 5190 terminated.
23-05-23 11:03:47.486 [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_serve

[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 4730 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 4357...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 4357 terminated.


In [ ]:
# | export


async def run_fastkafka_server(num_workers: int, app: str, kafka_broker: str) -> None:
    """
    Runs the FastKafka server with multiple worker processes.

    Args:
        num_workers (int): Number of FastKafka instances to run.
        app (str): Input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.
        kafka_broker (str): Kafka broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastKafka class.
    """
    loop = asyncio.get_event_loop()

    HANDLED_SIGNALS = (
        signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
        signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
    )
    if platform.system() == "Windows":
            HANDLED_SIGNALS = (*HANDLED_SIGNALS, signal.SIGBREAK) # type: ignore

    d = {"should_exit": False}

    def handle_windows_exit(
        signum: int, frame: Optional[FrameType], d: Dict[str, bool] = d
    ) -> None:
        d["should_exit"] = True

    def handle_exit(sig: int, d: Dict[str, bool] = d) -> None:
        d["should_exit"] = True

    for sig in HANDLED_SIGNALS:
        if platform.system() == "Windows":
            signal.signal(sig, handle_windows_exit)
        else:
            loop.add_signal_handler(sig, handle_exit, sig)

    async with asyncer.create_task_group() as tg:
        args = [
            "run_fastkafka_server_process",
            "--kafka-broker",
            kafka_broker,
            app,
        ]
        tasks = [
            tg.soonify(asyncio.create_subprocess_exec)(
                *args,
                stdout=asyncio.subprocess.PIPE,
                stdin=asyncio.subprocess.PIPE,
            )
            for i in range(num_workers)
        ]

    procs = [task.value for task in tasks]

    async def log_output(
        output: Optional[asyncio.StreamReader], pid: int, d: Dict[str, bool] = d
    ) -> None:
        if output is None:
            raise RuntimeError("Expected StreamReader, got None. Is stdout piped?")
        while not output.at_eof():
            outs = await output.readline()
            if outs != b"":
                typer.echo(f"[{pid:03d}]: " + outs.decode("utf-8"), nl=False)

    async with asyncer.create_task_group() as tg:
        for proc in procs:
            tg.soonify(log_output)(proc.stdout, proc.pid)

        while not d["should_exit"]:
            await asyncio.sleep(0.2)

        typer.echo("Starting process cleanup, this may take a few seconds...")
        for proc in procs:
            tg.soonify(terminate_asyncio_process)(proc)

    for proc in procs:
        output, _ = await proc.communicate()
        if output:
            typer.echo(f"[{proc.pid:03d}]: " + output.decode("utf-8"), nl=False)

    returncodes = [proc.returncode for proc in procs]
    if not returncodes == [0] * len(procs):
        typer.secho(
            f"Return codes are not all zero: {returncodes}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(1)

In [ ]:
# | export


@contextmanager
def run_in_process(
    target: Callable[..., Any]
) -> Generator[multiprocessing.Process, None, None]:
    """
    Runs the target function in a separate process.

    Args:
        target (Callable[..., Any]): The function to run in a separate process.

    Yields:
        Generator[multiprocessing.Process, None, None]: A generator that yields the process object.
    """
    p = multiprocessing.Process(target=target)
    try:
        p.start()
        yield p
    except Exception as e:
        print(f"Exception raised {e=}")
    finally:
        p.terminate()
        p.join()

In [ ]:
# | notest

listener_port = 10000
async with ApacheKafkaBroker(listener_port=listener_port) as bootstrap_server:
    os.environ["KAFKA_HOSTNAME"], os.environ["KAFKA_PORT"] = bootstrap_server.split(":")

    with generate_app_in_tmp() as app:

        def run_fastkafka_server_test():
            asyncio.run(run_fastkafka_server(4, app, "localhost"))

        with run_in_process(run_fastkafka_server_test) as p:
            sleep(15)

        assert p.exitcode == 0, p.exitcode
        p.close()

print("ok")

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
stdout=, stderr=, returncode=1
[INFO] fastkafka._testing.apache_kafka_broker: zookeeper startup falied, generating a new port and retrying...
[INFO] fastkafka._testing.apache_kafka_broker: port=44667
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:10000
[6427]: 23-05-23 11:04:00.066 [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to '127.0.0.1:10000'
[6427]: 23-05-23 11:04:00.067 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:10000'}'
[6427]: 23-05-23 11:04:00.090 [INFO] fastkafka._application.app: _create_producer() : created producer using the config:

[6423]: 23-05-23 11:04:00.163 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[6423]: 23-05-23 11:04:00.170 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[6423]: 23-05-23 11:04:00.170 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[6423]: 23-05-23 11:04:00.175 [ERROR] aiokafka.cluster: Topic realitime_data not found in cluster metadata
[6425]: 23-05-23 11:04:00.157 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[6425]: 23-05-23 11:04:00.157 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'group_id': '127.0.0.1:10000_group', 'auto_offset_reset': 'earliest', 'bootstrap_servers': '127.0.0.1:10000', 'max_poll_records': 100}
[6425]: 23-05-23 11:04:00.158 [INFO] fastkafka._com

[6425]: 23-05-23 11:04:00.391 [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[6425]: 23-05-23 11:04:00.391 [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[6429]: 23-05-23 11:04:00.394 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[6429]: 23-05-23 11:04:00.396 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[6429]: 23-05-23 11:04:00.396 [WARNING] aiokafka.cluster: Topic training_data is not available during auto-create initialization
[6429]: 23-05-23 11:04:00.398 [WARNING] aiokafka.cluster: Topic realitime_data is not available during auto-create initialization
[6423]: 23-05-23 11:04:00.406 [ERROR] aiokafka.consumer.group_coordinator: Group Coordinator Request failed: [Error 15] CoordinatorNotAvailableError
[6423]: 23-05-23 11:04:00.408 [WARN

[6423]: 23-05-23 11:04:00.722 [INFO] aiokafka.consumer.group_coordinator: Discovered coordinator 0 for group 127.0.0.1:10000_group
[6423]: 23-05-23 11:04:00.722 [INFO] aiokafka.consumer.group_coordinator: Revoking previously assigned partitions set() for group 127.0.0.1:10000_group
[6423]: 23-05-23 11:04:00.722 [INFO] aiokafka.consumer.group_coordinator: (Re-)joining group 127.0.0.1:10000_group
[6427]: 23-05-23 11:04:00.724 [INFO] aiokafka.consumer.group_coordinator: Joined group '127.0.0.1:10000_group' (generation 1) with member_id aiokafka-0.8.0-7c8d991e-d7e0-42e8-b585-b99081610a0c
[6423]: 23-05-23 11:04:00.725 [INFO] aiokafka.consumer.group_coordinator: Joined group '127.0.0.1:10000_group' (generation 1) with member_id aiokafka-0.8.0-c9ff99ac-f1bc-4519-8f52-c983cd8d1678
[6423]: 23-05-23 11:04:00.725 [INFO] aiokafka.consumer.group_coordinator: Elected group leader -- performing partition assignments using roundrobin
[6425]: 23-05-23 11:04:00.726 [INFO] aiokafka.consumer.group_coordin

[6425]: 23-05-23 11:04:15.735 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[6425]: 23-05-23 11:04:15.736 [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[6425]: 23-05-23 11:04:15.736 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[6425]: 23-05-23 11:04:15.736 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[6423]: 23-05-23 11:04:15.736 [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[6423]: 23-05-23 11:04:15.736 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[6423]: 23-05-23 11:04:15.736 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[6423]: 23-05-23 11:04:15.736 [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
[6423]: 23-05-23 11:04:15.736 [INFO] fastkafka._components.aiokafka_consumer_loop: 